In [1]:
# First, import some libraries, functions or classes. I will use the CIFAR10 dataset. 
import torch
import torch.nn as nn
import torch.utils.data
from torch.autograd import Variable
import torch.optim as optim

import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [2]:
imageSize = 64
dataroot = './dataset'
batchSize = 64
niter = 25
lr = 0.0002
beta1 = 0.5
workers = 2
outf = './output'

In [3]:
nz = 100 # size of the latent z vector
ngf = 64
ndf = 64
nc = 3

In [4]:
# dataset class from the torchvision library
dataset = dset.CIFAR10(root=dataroot,download=True,
                       transform=transforms.Compose([
                           transforms.Scale(imageSize),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                       ]))
# DataLoader class constructs iterators over the dataset
dataloader = torch.utils.data.DataLoader(dataset=dataset,batch_size=batchSize,
                                        shuffle=True,num_workers=int(workers))

Files already downloaded and verified


In [5]:
# custom weights initialization called on netG and netD
def weights_init(m):
    # m represents a module
    # __class__和__name__是类的自带属性
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [6]:
class _netG(nn.Module):
    # deconvolutional generator
    # nn.Module是容器，是所有网络模块的基类
    # _netG也是一个容器,也是一个模块
    # torch.nn中的容器一共有４个：Module, Sequential, Modulelist, ParameterList
    def __init__(self):
        # initialzation method of class nn.Module, 子类继承父类的方法，但不代表实例初始化时运行父类的方法
        # super 相当于将其超越到父类，然后执行父类的init方法；与nn.Module.__init__()相比不用显式包含父类名
        super(_netG, self).__init__() 
        # Sequential也是一个网络容器，也是一个模块，包含的各层如卷积层也是一个独立的模块
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        output = self.main(input)
        return output

In [7]:
netG = _netG()
# apply是Module类的一个方法，递归应用到每一个module
# 可详见nn.Module的定义
netG.apply(weights_init)
print(netG)

_netG (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU (inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh ()
  )
)


In [8]:
class _netD(nn.Module):
    # convolutional discriminator
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)

        return output.view(-1, 1)

In [9]:
netD = _netD()
netD.apply(weights_init)
print(netD)

_netD (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid ()
  )
)


A criterion that measures the Binary Cross Entropy between the target and the ouput:
$$loss(o,t) = -1/n\sum_i(t[i]*log(o[i]) + (1 - t[i]) * log(1 - o[i]))$$

In [10]:
criterion = nn.BCELoss()

In [11]:
input = torch.FloatTensor(batchSize, 3, imageSize, imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
label = torch.FloatTensor(batchSize)
real_label = 1
fake_label = 0

In [12]:
cuda = 1
if cuda:
    netD.cuda()
    netG.cuda()
    criterion.cuda()
    input, label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

In [13]:
input = Variable(input)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)

In [14]:
# setup optimizer
#　参数是网络的参数
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [15]:
for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # 相当于超参数k＝１，D和G交替更新
        # train with real
        netD.zero_grad() # 梯度首先清零
        real_cpu, _ = data # 真实数据
        batch_size = real_cpu.size(0)
        input.data.resize_(real_cpu.size()).copy_(real_cpu)
        label.data.resize_(batch_size).fill_(real_label)　# 对于生成器的训练，标签应该要正确

        output = netD(input)
        # 利用了Ｄ的更新目标以及Binary Cross Entropy (BCE) loss、real_label、fake_label的特殊形式
        errD_real = criterion(output, label)　# 判别器对真实数据的错误率
        errD_real.backward()
        D_x = output.data.mean() # 对真实数据的判断

        # train with fake
        noise.data.resize_(batch_size, nz, 1, 1)
        noise.data.normal_(0, 1)　# 更新noise
        fake = netG(noise)
        label.data.fill_(fake_label)　# 对于生成器的训练，标签应该要正确
        output = netD(fake.detach()) # 为什么要detach？
        # 利用了G的更新目标以及Binary Cross Entropy (BCE) loss、real_label、fake_label的特殊形式
        errD_fake = criterion(output, label)　# 判别器对假数据的错误率
        errD_fake.backward()
        D_G_z1 = output.data.mean()　# 对假数据的判断
        errD = errD_real + errD_fake　# 判别器的总错误率
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        # 生成器无中生有
        ###########################
        netG.zero_grad()
        # 训练生成器的目的是以假乱真，让判别器认为是真的，所以应该是假样本往真标签靠近
        label.data.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)　# 这里的假样本和判别器训练时用的是一样的，会不会有影响？
        errG = criterion(output, label)　# 这里的损失函数实际上把论文中log的常数项舍去了
        errG.backward()
        # D_G_z1和D_G_z2实际上是判别器对与同样的假样本在判别器一次更新前后的判断对比
        D_G_z2 = output.data.mean()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, niter, i, len(dataloader),
                 errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03d.png' % (outf, epoch),
                    normalize=True)

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outf, epoch))

[0/25][0/782] Loss_D: 1.9362 Loss_G: 5.8231 D(x): 0.4640 D(G(z)): 0.5760 / 0.0049
[0/25][1/782] Loss_D: 1.2401 Loss_G: 5.6160 D(x): 0.6354 D(G(z)): 0.4149 / 0.0047
[0/25][2/782] Loss_D: 0.9856 Loss_G: 6.3458 D(x): 0.7843 D(G(z)): 0.4149 / 0.0030
[0/25][3/782] Loss_D: 0.8314 Loss_G: 5.8351 D(x): 0.6887 D(G(z)): 0.2665 / 0.0041
[0/25][4/782] Loss_D: 0.8459 Loss_G: 5.7761 D(x): 0.6974 D(G(z)): 0.2743 / 0.0040
[0/25][5/782] Loss_D: 1.0213 Loss_G: 7.7753 D(x): 0.7881 D(G(z)): 0.4428 / 0.0006
[0/25][6/782] Loss_D: 0.7511 Loss_G: 6.8095 D(x): 0.7183 D(G(z)): 0.1849 / 0.0014
[0/25][7/782] Loss_D: 0.9118 Loss_G: 8.1921 D(x): 0.7599 D(G(z)): 0.3863 / 0.0004
[0/25][8/782] Loss_D: 0.6382 Loss_G: 7.6948 D(x): 0.7632 D(G(z)): 0.2131 / 0.0007
[0/25][9/782] Loss_D: 0.8090 Loss_G: 8.6078 D(x): 0.8068 D(G(z)): 0.3040 / 0.0003
[0/25][10/782] Loss_D: 0.6484 Loss_G: 8.2721 D(x): 0.7565 D(G(z)): 0.2131 / 0.0004
[0/25][11/782] Loss_D: 0.7879 Loss_G: 9.5035 D(x): 0.7735 D(G(z)): 0.3200 / 0.0001
[0/25][12/782]

[0/25][99/782] Loss_D: 0.3284 Loss_G: 4.1347 D(x): 0.8260 D(G(z)): 0.0281 / 0.0735
[0/25][100/782] Loss_D: 1.6084 Loss_G: 7.4531 D(x): 0.9408 D(G(z)): 0.5190 / 0.0072
[0/25][101/782] Loss_D: 0.5518 Loss_G: 6.8657 D(x): 0.7369 D(G(z)): 0.0904 / 0.0073
[0/25][102/782] Loss_D: 0.1749 Loss_G: 5.9267 D(x): 0.9167 D(G(z)): 0.0683 / 0.0071
[0/25][103/782] Loss_D: 0.2156 Loss_G: 5.0611 D(x): 0.9473 D(G(z)): 0.1292 / 0.0105
[0/25][104/782] Loss_D: 0.2227 Loss_G: 5.5859 D(x): 0.9384 D(G(z)): 0.1305 / 0.0063
[0/25][105/782] Loss_D: 0.3405 Loss_G: 4.1577 D(x): 0.8203 D(G(z)): 0.0897 / 0.0265
[0/25][106/782] Loss_D: 0.4810 Loss_G: 6.5374 D(x): 0.9141 D(G(z)): 0.2721 / 0.0022
[0/25][107/782] Loss_D: 0.2515 Loss_G: 5.1124 D(x): 0.8512 D(G(z)): 0.0409 / 0.0085
[0/25][108/782] Loss_D: 0.3900 Loss_G: 2.9032 D(x): 0.7911 D(G(z)): 0.0758 / 0.0683
[0/25][109/782] Loss_D: 0.7596 Loss_G: 8.8236 D(x): 0.9771 D(G(z)): 0.4939 / 0.0003
[0/25][110/782] Loss_D: 1.4844 Loss_G: 4.6027 D(x): 0.3861 D(G(z)): 0.0044 / 

[0/25][197/782] Loss_D: 0.2360 Loss_G: 4.8349 D(x): 0.8363 D(G(z)): 0.0097 / 0.0105
[0/25][198/782] Loss_D: 0.5339 Loss_G: 10.3398 D(x): 0.9014 D(G(z)): 0.3000 / 0.0001
[0/25][199/782] Loss_D: 0.2505 Loss_G: 8.6872 D(x): 0.8330 D(G(z)): 0.0031 / 0.0007
[0/25][200/782] Loss_D: 0.0962 Loss_G: 5.6402 D(x): 0.9446 D(G(z)): 0.0183 / 0.0077
[0/25][201/782] Loss_D: 0.5602 Loss_G: 13.4079 D(x): 0.9821 D(G(z)): 0.3718 / 0.0000
[0/25][202/782] Loss_D: 0.2971 Loss_G: 12.9161 D(x): 0.8031 D(G(z)): 0.0001 / 0.0000
[0/25][203/782] Loss_D: 0.1143 Loss_G: 9.7148 D(x): 0.9114 D(G(z)): 0.0002 / 0.0001
[0/25][204/782] Loss_D: 0.0791 Loss_G: 5.1074 D(x): 0.9411 D(G(z)): 0.0072 / 0.0140
[0/25][205/782] Loss_D: 0.4292 Loss_G: 8.9594 D(x): 0.9894 D(G(z)): 0.2766 / 0.0003
[0/25][206/782] Loss_D: 0.0661 Loss_G: 8.8085 D(x): 0.9484 D(G(z)): 0.0097 / 0.0008
[0/25][207/782] Loss_D: 0.2818 Loss_G: 5.4458 D(x): 0.8361 D(G(z)): 0.0121 / 0.0121
[0/25][208/782] Loss_D: 0.2140 Loss_G: 6.3874 D(x): 0.9646 D(G(z)): 0.137

[0/25][295/782] Loss_D: 0.1371 Loss_G: 9.7692 D(x): 0.9136 D(G(z)): 0.0027 / 0.0003
[0/25][296/782] Loss_D: 0.1738 Loss_G: 5.6267 D(x): 0.9061 D(G(z)): 0.0120 / 0.0062
[0/25][297/782] Loss_D: 0.9388 Loss_G: 18.6964 D(x): 0.9673 D(G(z)): 0.4674 / 0.0000
[0/25][298/782] Loss_D: 3.9374 Loss_G: 15.6032 D(x): 0.1332 D(G(z)): 0.0000 / 0.0001
[0/25][299/782] Loss_D: 0.1363 Loss_G: 12.7482 D(x): 0.9021 D(G(z)): 0.0013 / 0.0015
[0/25][300/782] Loss_D: 0.0457 Loss_G: 8.9763 D(x): 0.9958 D(G(z)): 0.0339 / 0.0032
[0/25][301/782] Loss_D: 0.1733 Loss_G: 7.1446 D(x): 0.9822 D(G(z)): 0.1192 / 0.0030
[0/25][302/782] Loss_D: 0.9852 Loss_G: 15.3422 D(x): 0.9868 D(G(z)): 0.4609 / 0.0000
[0/25][303/782] Loss_D: 0.3012 Loss_G: 15.2172 D(x): 0.8081 D(G(z)): 0.0001 / 0.0000
[0/25][304/782] Loss_D: 1.2898 Loss_G: 7.2838 D(x): 0.4399 D(G(z)): 0.0003 / 0.0090
[0/25][305/782] Loss_D: 0.2611 Loss_G: 5.3149 D(x): 0.9896 D(G(z)): 0.1692 / 0.0299
[0/25][306/782] Loss_D: 0.6616 Loss_G: 11.1431 D(x): 0.9927 D(G(z)): 0.

[0/25][393/782] Loss_D: 0.3761 Loss_G: 4.0381 D(x): 0.7625 D(G(z)): 0.0430 / 0.0359
[0/25][394/782] Loss_D: 0.3407 Loss_G: 3.0456 D(x): 0.8407 D(G(z)): 0.1231 / 0.0971
[0/25][395/782] Loss_D: 0.4273 Loss_G: 4.2144 D(x): 0.9192 D(G(z)): 0.2454 / 0.0230
[0/25][396/782] Loss_D: 0.3144 Loss_G: 3.9576 D(x): 0.8756 D(G(z)): 0.1229 / 0.0259
[0/25][397/782] Loss_D: 0.6486 Loss_G: 3.0958 D(x): 0.7072 D(G(z)): 0.1508 / 0.1132
[0/25][398/782] Loss_D: 0.4844 Loss_G: 5.1304 D(x): 0.9339 D(G(z)): 0.2971 / 0.0138
[0/25][399/782] Loss_D: 0.6313 Loss_G: 2.2322 D(x): 0.6506 D(G(z)): 0.0475 / 0.1631
[0/25][400/782] Loss_D: 0.5113 Loss_G: 4.2409 D(x): 0.9365 D(G(z)): 0.3139 / 0.0262
[0/25][401/782] Loss_D: 0.2821 Loss_G: 4.0196 D(x): 0.8269 D(G(z)): 0.0660 / 0.0416
[0/25][402/782] Loss_D: 0.4303 Loss_G: 3.1439 D(x): 0.8221 D(G(z)): 0.1397 / 0.0817
[0/25][403/782] Loss_D: 0.3121 Loss_G: 3.9274 D(x): 0.9290 D(G(z)): 0.1865 / 0.0344
[0/25][404/782] Loss_D: 0.1766 Loss_G: 4.2709 D(x): 0.9094 D(G(z)): 0.0658 /

...

[24/25][344/782] Loss_D: 0.0173 Loss_G: 5.5035 D(x): 0.9990 D(G(z)): 0.0156 / 0.0081
[24/25][345/782] Loss_D: 0.0099 Loss_G: 5.7562 D(x): 0.9982 D(G(z)): 0.0081 / 0.0048
[24/25][346/782] Loss_D: 0.0132 Loss_G: 5.9230 D(x): 0.9945 D(G(z)): 0.0076 / 0.0053
[24/25][347/782] Loss_D: 0.0361 Loss_G: 5.6954 D(x): 0.9945 D(G(z)): 0.0286 / 0.0063
[24/25][348/782] Loss_D: 0.0313 Loss_G: 6.5396 D(x): 0.9725 D(G(z)): 0.0025 / 0.0027
[24/25][349/782] Loss_D: 0.0427 Loss_G: 5.1667 D(x): 0.9912 D(G(z)): 0.0327 / 0.0091
[24/25][350/782] Loss_D: 0.0073 Loss_G: 7.4869 D(x): 0.9941 D(G(z)): 0.0014 / 0.0011
[24/25][351/782] Loss_D: 0.0292 Loss_G: 5.3738 D(x): 0.9937 D(G(z)): 0.0223 / 0.0080
[24/25][352/782] Loss_D: 0.0332 Loss_G: 6.7464 D(x): 0.9697 D(G(z)): 0.0019 / 0.0020
[24/25][353/782] Loss_D: 0.0176 Loss_G: 6.2097 D(x): 0.9847 D(G(z)): 0.0018 / 0.0036
[24/25][354/782] Loss_D: 0.0263 Loss_G: 5.1795 D(x): 0.9968 D(G(z)): 0.0222 / 0.0089
[24/25][355/782] Loss_D: 0.0106 Loss_G: 7.5524 D(x): 0.9907 D(G(z

[24/25][441/782] Loss_D: 0.0777 Loss_G: 4.9101 D(x): 0.9537 D(G(z)): 0.0275 / 0.0146
[24/25][442/782] Loss_D: 0.0729 Loss_G: 4.6027 D(x): 0.9677 D(G(z)): 0.0359 / 0.0173
[24/25][443/782] Loss_D: 0.0671 Loss_G: 4.7797 D(x): 0.9764 D(G(z)): 0.0404 / 0.0163
[24/25][444/782] Loss_D: 0.1249 Loss_G: 3.9644 D(x): 0.9194 D(G(z)): 0.0299 / 0.0451
[24/25][445/782] Loss_D: 0.0546 Loss_G: 4.7564 D(x): 0.9856 D(G(z)): 0.0358 / 0.0161
[24/25][446/782] Loss_D: 0.1042 Loss_G: 6.4308 D(x): 0.9922 D(G(z)): 0.0882 / 0.0025
[24/25][447/782] Loss_D: 0.0772 Loss_G: 5.2632 D(x): 0.9479 D(G(z)): 0.0192 / 0.0111
[24/25][448/782] Loss_D: 0.1168 Loss_G: 4.5782 D(x): 0.9477 D(G(z)): 0.0543 / 0.0241
[24/25][449/782] Loss_D: 0.0343 Loss_G: 5.2098 D(x): 0.9913 D(G(z)): 0.0245 / 0.0115
[24/25][450/782] Loss_D: 0.0312 Loss_G: 5.4056 D(x): 0.9836 D(G(z)): 0.0142 / 0.0093
[24/25][451/782] Loss_D: 0.0136 Loss_G: 6.2183 D(x): 0.9925 D(G(z)): 0.0058 / 0.0048
[24/25][452/782] Loss_D: 0.0546 Loss_G: 5.1076 D(x): 0.9890 D(G(z

[24/25][538/782] Loss_D: 0.9733 Loss_G: 6.4741 D(x): 0.9921 D(G(z)): 0.5254 / 0.0027
[24/25][539/782] Loss_D: 0.3857 Loss_G: 3.7472 D(x): 0.7389 D(G(z)): 0.0184 / 0.0512
[24/25][540/782] Loss_D: 0.4179 Loss_G: 1.5826 D(x): 0.8071 D(G(z)): 0.1333 / 0.2765
[24/25][541/782] Loss_D: 0.5256 Loss_G: 5.2666 D(x): 0.9584 D(G(z)): 0.3268 / 0.0075
[24/25][542/782] Loss_D: 0.4829 Loss_G: 2.5175 D(x): 0.6883 D(G(z)): 0.0330 / 0.1110
[24/25][543/782] Loss_D: 0.4500 Loss_G: 1.5832 D(x): 0.7982 D(G(z)): 0.1203 / 0.2720
[24/25][544/782] Loss_D: 1.1620 Loss_G: 7.5173 D(x): 0.9721 D(G(z)): 0.5887 / 0.0011
[24/25][545/782] Loss_D: 1.8428 Loss_G: 1.2732 D(x): 0.2305 D(G(z)): 0.0035 / 0.3726
[24/25][546/782] Loss_D: 1.1444 Loss_G: 4.6870 D(x): 0.9724 D(G(z)): 0.5723 / 0.0161
[24/25][547/782] Loss_D: 0.3742 Loss_G: 3.7924 D(x): 0.7688 D(G(z)): 0.0520 / 0.0387
[24/25][548/782] Loss_D: 0.3235 Loss_G: 2.4723 D(x): 0.8112 D(G(z)): 0.0709 / 0.1310
[24/25][549/782] Loss_D: 0.3281 Loss_G: 3.7572 D(x): 0.9646 D(G(z

[24/25][635/782] Loss_D: 0.1156 Loss_G: 4.4330 D(x): 0.9605 D(G(z)): 0.0684 / 0.0196
[24/25][636/782] Loss_D: 0.2213 Loss_G: 3.5903 D(x): 0.8548 D(G(z)): 0.0390 / 0.0543
[24/25][637/782] Loss_D: 0.1644 Loss_G: 3.9842 D(x): 0.9891 D(G(z)): 0.1303 / 0.0314
[24/25][638/782] Loss_D: 0.1233 Loss_G: 4.2147 D(x): 0.9317 D(G(z)): 0.0407 / 0.0254
[24/25][639/782] Loss_D: 0.0619 Loss_G: 4.4576 D(x): 0.9731 D(G(z)): 0.0317 / 0.0247
[24/25][640/782] Loss_D: 0.1689 Loss_G: 4.3735 D(x): 0.9765 D(G(z)): 0.1221 / 0.0187
[24/25][641/782] Loss_D: 0.1493 Loss_G: 3.9399 D(x): 0.8981 D(G(z)): 0.0300 / 0.0310
[24/25][642/782] Loss_D: 0.0944 Loss_G: 3.7774 D(x): 0.9366 D(G(z)): 0.0249 / 0.0394
[24/25][643/782] Loss_D: 0.2797 Loss_G: 6.7703 D(x): 0.9979 D(G(z)): 0.2169 / 0.0020
[24/25][644/782] Loss_D: 0.3355 Loss_G: 5.1692 D(x): 0.7480 D(G(z)): 0.0070 / 0.0111
[24/25][645/782] Loss_D: 0.2758 Loss_G: 1.5992 D(x): 0.8715 D(G(z)): 0.1021 / 0.2754
[24/25][646/782] Loss_D: 0.5175 Loss_G: 8.7810 D(x): 0.9949 D(G(z

[24/25][732/782] Loss_D: 0.0291 Loss_G: 6.0554 D(x): 0.9765 D(G(z)): 0.0051 / 0.0050
[24/25][733/782] Loss_D: 0.0449 Loss_G: 4.9486 D(x): 0.9940 D(G(z)): 0.0365 / 0.0140
[24/25][734/782] Loss_D: 0.0149 Loss_G: 6.2755 D(x): 0.9891 D(G(z)): 0.0039 / 0.0034
[24/25][735/782] Loss_D: 0.0495 Loss_G: 4.7282 D(x): 0.9898 D(G(z)): 0.0368 / 0.0166
[24/25][736/782] Loss_D: 0.0222 Loss_G: 5.4658 D(x): 0.9858 D(G(z)): 0.0078 / 0.0068
[24/25][737/782] Loss_D: 0.0169 Loss_G: 5.6686 D(x): 0.9914 D(G(z)): 0.0082 / 0.0074
[24/25][738/782] Loss_D: 0.0358 Loss_G: 4.7835 D(x): 0.9804 D(G(z)): 0.0153 / 0.0153
[24/25][739/782] Loss_D: 0.0167 Loss_G: 5.4084 D(x): 0.9923 D(G(z)): 0.0087 / 0.0085
[24/25][740/782] Loss_D: 0.0617 Loss_G: 5.2565 D(x): 0.9990 D(G(z)): 0.0573 / 0.0077
[24/25][741/782] Loss_D: 0.0250 Loss_G: 5.8314 D(x): 0.9846 D(G(z)): 0.0092 / 0.0057
[24/25][742/782] Loss_D: 0.0505 Loss_G: 4.2987 D(x): 0.9689 D(G(z)): 0.0176 / 0.0212
[24/25][743/782] Loss_D: 0.0441 Loss_G: 4.7548 D(x): 0.9893 D(G(z

判别器太强势，影响生成器学习